Engenharia de Software - PUC RJ
Notebook MVP da Sprint 4.

Este notebook realiza o treinamento de Chabot desenvolvido para a implementação de FAQs via web.

In [21]:
# Imports
import json
import pickle

# Bibliotecas de NLP
import nltk
nltk.download('rslp')
nltk.download('punkt')
stemmer = nltk.stem.RSLPStemmer()

# Tensorflow
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.optimizers import SGD
import random

[nltk_data] Downloading package rslp to /root/nltk_data...
[nltk_data]   Package rslp is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [22]:
# Inicialização das variáveis
words = []
classes = []
documents = []
ignore_words = ['?', '!', ',', ';']

In [23]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [24]:
# Importação dos Intents
#
# Nota >> O arquivo com as intents deve estar disponível no google drive na pasta chatbot!!!
#
data_file = open('/content/drive/MyDrive/chatbot/intents.json').read()
intents = json.loads(data_file)


for intent in intents['intents']:
    for pattern in intent['patterns']:
        # Tokenize cada palavra da sentença
        w = nltk.word_tokenize(pattern)
        # aciciona as palavras em um lista.
        words.extend(w)
        # add to documents in our corpus
        documents.append((w, intent['tag']))
        # add to our classes list
        if intent['tag'] not in classes:
            classes.append(intent['tag'])

# stem and lower cada palavra
words = [stemmer.stem(w.lower()) for w in words if w not in ignore_words]
words = sorted(list(set(words)))

# remove os duplicados
classes = sorted(list(set(classes)))

print(f"quantidade de documents >>  {len(documents)}")
print(f"quantidade de classes >>  {len(classes)}")
print(f"quantidade unique stemmed words >> {len(words)}")

quantidade de documents >>  65
quantidade de classes >>  13
quantidade unique stemmed words >> 95


In [25]:
# Gera o arquivo Words
file = open('words.pkl','wb')
pickle.dump(words,file)
file.close()

# Gera o arquivo Classes
file = open('classes.pkl','wb')
pickle.dump(classes,file)
file.close()

In [26]:
# cria training data
training = []
output = []

# cria um array vazio para output
output_empty = [0] * len(classes)

# training set, bag of words for casa sentença
for doc in documents:
    bag = []
    pattern_words = doc[0]
    pattern_words = [stemmer.stem(word.lower()) for word in pattern_words]
    for w in words:
        bag.append(1) if w in pattern_words else bag.append(0)
    output_row = list(output_empty)
    output_row[classes.index(doc[1])] = 1
    training.append([bag, output_row])

# shuffle em np.array
random.shuffle(training)
training = np.array(training, dtype="object")

# cria o lista de treinamento e teste
train_x = list(training[:,0])
train_y = list(training[:,1])

In [27]:
# Cria o Modelo
model = Sequential()
model.add(Dense(512, input_shape=(len(train_x[0]),), activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(len(train_y[0]), activation='softmax'))

# Copila o modelo
sgd = SGD(lr=0.01, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])

# fitting  e salva o modelo
hist = model.fit(np.array(train_x), np.array(train_y), epochs=1000, batch_size=1000, verbose=1)
model.save('chatbot_model.h5', hist)
print("Modelo Criado com sucesso.")

Epoch 1/1000
1/1 [==============================] - 1s 717ms/step - loss: 2.5960 - accuracy: 0.0769
Epoch 2/1000
1/1 [==============================] - 0s 17ms/step - loss: 2.5604 - accuracy: 0.0769
Epoch 3/1000
1/1 [==============================] - 0s 17ms/step - loss: 2.5873 - accuracy: 0.0923
Epoch 4/1000
1/1 [==============================] - 0s 15ms/step - loss: 2.5310 - accuracy: 0.1846
Epoch 5/1000
1/1 [==============================] - 0s 15ms/step - loss: 2.5439 - accuracy: 0.1231
Epoch 6/1000
1/1 [==============================] - 0s 15ms/step - loss: 2.5606 - accuracy: 0.1077
Epoch 7/1000
1/1 [==============================] - 0s 14ms/step - loss: 2.5465 - accuracy: 0.1231
Epoch 8/1000
1/1 [==============================] - 0s 15ms/step - loss: 2.5826 - accuracy: 0.0462
Epoch 9/1000
1/1 [==============================] - 0s 15ms/step - loss: 2.5343 - accuracy: 0.1231
Epoch 10/1000
1/1 [==============================] - 0s 17ms/step - loss: 2.5291 - accuracy: 0.1077
Epoch 11

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3079: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


# New Section

In [28]:
# Salva o modelo no google drive
!cp chatbot_model.h5 /content/drive/MyDrive/chatbot